### Goal
This notebook describes how to create a simple generative adversarial network (GAN). The aim here is to build and train a GAN that can generate hand-written images of digits (0-9). The code is implmented using keras/Tensorflow 2.

In [1]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from tqdm import tqdm
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, BatchNormalization, Conv2D, Dense, Flatten, Add,GlobalAveragePooling2D,Conv2DTranspose
from tensorflow.keras.models import  Sequential, Model, load_model

import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from keras import callbacks


In [2]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
INPUT_SHAPE = (127, 127)
n_nodes = 128 * 7 * 7 
NOISE_DIM =100

In [4]:
####Create a custom model that integrates the residual blocks ####
class GeneratorModel(tf.keras.Model):
    
    def __init__(self, fillters=[ 128, 128,1], kernel_sizes=[4, 4,7], strides=[2, 2,2], input_shape=NOISE_DIM):
                
        super(GeneratorModel, self).__init__()
        self.dn1 = Dense(n_nodes, input_dim=NOISE_DIM, activation='selu')
        self.rs1 = Reshape((7, 7, 128))
        self.bn1 = BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2DTranspose(fillters[0], (kernel_sizes[0], kernel_sizes[0]), 
                                               strides=(strides[0], strides[0]),padding="same", activation='selu')
        self.bn2 = BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2DTranspose(fillters[1], (kernel_sizes[1], kernel_sizes[1]), 
                                               strides=(strides[1], strides[1]), padding="same",activation='selu')
        self.bn3 = BatchNormalization()
        self.conv3 = tf.keras.layers.Conv2D(fillters[2], (kernel_sizes[2], kernel_sizes[2]), 
                                               padding="same", activation='sigmoid')
        
        
    def call(self, inputs):
        x = self.dn1(inputs)
        x = self.rs1(x)
        x = self.bn1(x)
        x = self.conv1(x)
        x = self.bn2(x)
        x = self.conv2(x)
        x = self.bn3(x)
        x = self.conv3(x)        
        return x

In [5]:
####Create a custom model that integrates the residual blocks ####
class DiscriminatorModel(tf.keras.Model):
    
    def __init__(self, fillters=[ 64, 64], kernel_sizes=[3, 3], strides=[2, 2], input_shape=INPUT_SHAPE):
                
        super(DiscriminatorModel, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(fillters[0], (kernel_sizes[0], kernel_sizes[0]), 
                                               strides=(strides[0], strides[0]), padding="same",activation='selu')
        self.do1 = Dropout(0.4)
        self.bn1 = BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(fillters[1], (kernel_sizes[1], kernel_sizes[1]), 
                                               strides=(strides[1], strides[1]),padding="same", activation='selu')
        self.do2 = Dropout(0.4)
        self.bn2 = BatchNormalization()
        
        self.flatten = Flatten()
        self.fc = Dense(1, activation='sigmoid')
        
    def call(self, inputs, training=False):       
        
        x = self.conv1(inputs)
        x = self.do1(x)
        x = self.conv2(x)
        x = self.do2(x)
        x = self.flatten(x)        
        x = self.fc(x)        
        return x
    
test_model = DiscriminatorModel()

In [6]:
# define the combined generator and discriminator model, for updating the generator
def DefineGan(g_model, d_model):
    # make weights in the discriminator not trainable
    for layer in d_model.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False
            
    model = Sequential()

    model.add(g_model)
    model.add(d_model)
    return model

# generate points in latent space as input for the generator
def Generate_Noise(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [7]:
# select real samples
def Sample_real_data(dataset, n_samples):
    ix = randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = ones((n_samples, 1))
    return X, y

# use the generator to generate n fake examples, with class labels
def Sample_fake_data(g_model, latent_dim, n_samples):
    x_input = Generate_Noise(latent_dim, n_samples)
    X = g_model.predict(x_input)
    #print(X.shape)
    y = zeros((n_samples, 1))
    return X, y

# create and save a plot of generated images (reversed grayscale)
def save_plot(examples, epoch, n=10):
    for i in range(n * n):
        pyplot.subplot(n, n, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
    filename = 'results/Generated_plot_2_e%03d.png' % (epoch+1)
    pyplot.savefig(filename)
    pyplot.close()

# evaluate the discriminator, plot generated images, save generator model
def PrintProgress(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    X_real, y_real = Sample_real_data(dataset, n_samples)
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    x_fake, y_fake = Sample_fake_data(g_model, latent_dim, n_samples)
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    save_plot(x_fake, epoch)
    filename = 'results/generator_model_%03d.h5' % (epoch + 1)
    g_model.save_weights(filename)

# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=256):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    d_hist,  g_hist, da_hist, ga_hist = list(), list(), list(), list()
    for i in tqdm(range(n_epochs*bat_per_epo)):
        X_real, y_real = Sample_real_data(dataset, int(n_batch / 2))
        X_fake, y_fake = Sample_fake_data(g_model, latent_dim, int(n_batch / 2))
        #print(X_fake.shape, X_real.shape)
        X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))

        #d_loss, d_acc = d_model.train_on_batch(X, y)
        d_loss1, d_acc1 = d_model.train_on_batch(X_real, y_real)
        d_loss2, d_acc2 = d_model.train_on_batch(X_fake, y_fake)

        # train the genrator by genrating using fake samples but real like lables 
        X_gan = Generate_Noise(latent_dim, n_batch)
        y_gan = ones((n_batch, 1))
        g_loss,_ = gan_model.train_on_batch(X_gan, y_gan)

            
            
        if (i+1) % bat_per_epo*5 == 0:
            print('>%d,  dis_loss=%.3f, gen_loss=%.3f' % (i+1, (d_loss1+d_loss2), g_loss))
            PrintProgress(i, g_model, d_model, dataset, latent_dim)


In [8]:
# load and prepare mnist training images
def load_real_samples():
    (trainX, _), (_, _) = load_data()
    X = expand_dims(trainX, axis=-1)
    X = X.astype('float32')
    X = X / 255.0
    return X


def plot_history(d1_hist, g_hist, a1_hist, a2_hist):
    # plot loss
    pyplot.subplot(2, 1, 1)
    pyplot.plot(d1_hist, label='dis-loss')
    pyplot.plot(g_hist, label='gen-loss')
    pyplot.legend()
    
    # plot discriminator accuracy
    pyplot.subplot(2, 1, 2)
    pyplot.plot(a1_hist, label='acc-real')
    pyplot.plot(a2_hist, label='acc-fake')
    pyplot.legend()
    
    # save plot to file
    pyplot.savefig('results/plot_line_plot_loss.png')
    pyplot.close()

In [9]:
generator_optimizer = Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.9)
discriminator_optimizer = Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.9)

d_model = DiscriminatorModel()
d_model.compile(loss='binary_crossentropy', optimizer=discriminator_optimizer, metrics=['accuracy'])

g_model = GeneratorModel(input_shape = NOISE_DIM)

# create the gan
gan_model = DefineGan(g_model, d_model)
gan_model.compile(loss='binary_crossentropy', optimizer=generator_optimizer, metrics=['accuracy'])

# load image data
dataset = load_real_samples()

# train model
train(g_model, d_model, gan_model, dataset, NOISE_DIM)

  1%|▊                                                                             | 233/23400 [00:31<45:20,  8.52it/s]

>234,  dis_loss=1.455, gen_loss=0.442
>Accuracy real: 92%, fake: 3%


  2%|█▌                                                                            | 467/23400 [01:05<55:18,  6.91it/s]

>468,  dis_loss=1.426, gen_loss=0.698
>Accuracy real: 45%, fake: 46%


  3%|██▎                                                                           | 701/23400 [01:39<52:50,  7.16it/s]

...

 99%|███████████████████████████████████████████████████████████████████████████▏| 23165/23400 [53:03<00:30,  7.75it/s]

>23166,  dis_loss=1.397, gen_loss=0.693
>Accuracy real: 58%, fake: 44%


100%|███████████████████████████████████████████████████████████████████████████▉| 23399/23400 [53:35<00:00,  7.78it/s]

>23400,  dis_loss=1.388, gen_loss=0.695
>Accuracy real: 58%, fake: 44%


100%|████████████████████████████████████████████████████████████████████████████| 23400/23400 [53:37<00:00,  7.27it/s]
